1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import re
import json

In [2]:
def comp_distribution(text = ''):
    res = {'min_compensation': None, 'max_compensation': None, 'currency': None}
    if text == '':
        return res
    text = re.sub(r'[\.\s]', "", text)
    text = re.split('(\d+)', text)
    res['currency'] = text.pop()
    if len(text) == 4:
        res['min_compensation'] = int(text[1])
        res['max_compensation'] = int(text[3])
    if len(text) == 2:
        if text[0].lower() == 'от':
            res['min_compensation'] = int(text[1])
        if text[0].lower() == 'до':
            res['max_compensation'] = int(text[1])
    return res

#print(comp_distribution('от 4 000 USD	'))

In [3]:
def get_pos_id (link):
    return re.search(r'(\d+)', link).group()
    
#print(get_pos_id('https://hh.ru/vacancy/37674249?query=python'))

In [4]:
columns = ['_id', 'Position_name', 'Employer_name', 'City', 'Position_link', 'Source', 'Min_compensation',
           'Max_compensation', 'Currency']
df = pd.DataFrame(columns=columns)

In [5]:
main_link = 'https://hh.ru'

header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}

params = {'L_save_area': 'true',
          'clusters': 'true',
          'enable_snippets': 'true',
          'text': 'Python',
          'showClusters': 'true'}

In [6]:
response = requests.get(main_link + '/search/vacancy', headers=header, params=params).text
soup = bs(response, 'lxml')

In [7]:
data_to_db = []
while True:
    
    vacancy_block = soup.find('div', {'class': 'vacancy-serp'})

    vacancy_list_premium = vacancy_block.find_all('div', {'data-qa': 'vacancy-serp__vacancy vacancy-serp__vacancy_premium'})
    for vacancy in vacancy_list_premium:
        vacancy_data = {}
        
        vacancy_data['Position_name'] = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'}).getText()
        vacancy_data['Employer_name'] = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'}).getText()
        vacancy_data['City'] = vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText()
        pos_link = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        vacancy_data['Position_link'] = pos_link
        vacancy_data['_id'] = main_link + '_' + get_pos_id(pos_link)                                 
        vacancy_data['Source'] = main_link
        compensation_data = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        compensation_data = comp_distribution(compensation_data)
        vacancy_data['Min_compensation'] = compensation_data['min_compensation']
        vacancy_data['Max_compensation'] = compensation_data['max_compensation']
        vacancy_data['Currency'] = compensation_data['currency']
        
        data_to_db.append(vacancy_data)
        df = df.append(pd.Series(vacancy_data, index=df.columns), ignore_index=True)
        
    vacancy_list = vacancy_block.find_all('div', {'data-qa': 'vacancy-serp__vacancy'})
    for vacancy in vacancy_list:

        vacancy_data['Position_name'] = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'}).getText()
        employer_name = vacancy.find('a', {'class': 'bloko-link bloko-link_secondary'})
        if employer_name:                        
            vacancy_data['Employer_name'] = employer_name.getText()
        else:
            vacancy_data['Employer_name'] = None
        vacancy_data['City'] =  vacancy.find('span', {'class': 'vacancy-serp-item__meta-info'}).getText()
        pos_link = vacancy.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        vacancy_data['Position_link'] = pos_link
        vacancy_data['_id'] = main_link + '_' + get_pos_id(pos_link)
        vacancy_data['Source'] = main_link
        compensation_data = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).getText()
        compensation_data = comp_distribution(compensation_data)
        vacancy_data['Min_compensation'] = compensation_data['min_compensation']
        vacancy_data['Max_compensation'] = compensation_data['max_compensation']
        vacancy_data['Currency'] = compensation_data['currency']
        
        data_to_db.append(vacancy_data)
        df = df.append(pd.Series(vacancy_data, index=df.columns), ignore_index=True)
    
    
    next_page_link_block = soup.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})

    if not next_page_link_block:
        break

    next_page_link = next_page_link_block['href']
    response = requests.get(main_link + next_page_link, headers=header).text
    soup = bs(response, 'lxml')

In [8]:
df.head(5)

,_id,Position_name,Employer_name,City,Position_link,Source,Min_compensation,Max_compensation,Currency
0,https://hh.ru_37765447,Скилловый Python Developer на бэкенд,Интеллектуальные системы,"Санкт-Петербург, Петроградская",https://hh.ru/vacancy/37765447?query=Python,https://hh.ru,250000,None,руб
1,https://hh.ru_37674249,Программист-разработчик python (midle),CATAPULTO.RU,Москва,https://hh.ru/vacancy/37674249?query=Python,https://hh.ru,100000,180000,руб
2,https://hh.ru_37461514,Python / Django fullstack программист,ООО Чистый Маркетинг,Москва,https://hh.ru/vacancy/37461514?query=Python,https://hh.ru,110000,180000,руб
3,https://hh.ru_37792411,Python Developer,Scand,Минск,https://hh.ru/vacancy/37792411?query=Python,https://hh.ru,None,None,None
4,https://hh.ru_37552725,Middle software developer (ML),ТОО Click Logistics,Алматы,https://hh.ru/vacancy/37552725?query=Python,https://hh.ru,300000,500000,KZT


In [9]:
df.describe()

,_id,Position_name,Employer_name,City,Position_link,Source,Min_compensation,Max_compensation,Currency
count,2000,2000,1999,2000,2000,2000,572,401,652
unique,1820,1315,947,298,1820,1,74,64,7
top,https://hh.ru_35715672,DevOps инженер,Сбербанк для экспертов,Москва,https://hh.ru/vacancy/35715672?query=Python,https://hh.ru,100000,150000,руб
freq,3,26,62,501,3,2000,66,58,593


In [10]:
path = '/Users/svetlanaskobeltcyna/PycharmProjects/Data_collection_methods/lesson2/'
df.to_csv(path + 'hh_vacancy_data.csv', index=False)

In [11]:
with open(path + 'vacancy_data.json', 'w', encoding = 'utf-8') as f:
    json.dump(data_to_db, f)

In [12]:
len(data_to_db)


2000